les pip :
spacy
spacy-lefff
collections
pandas
git+https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git

le reste :
spacy download fr_core_news_sm 

In [ ]:
import spacy 
from spacy_lefff import LefffLemmatizer
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
import pandas as pd
from collections import Counter

In [ ]:
sample_txt="Le malheureux Dantès désespère dans sa captivité jusqu'à songer au suicide. Il a la chance de faire la connaissance de l'abbé Faria, un autre prisonnier qui, voulant s'évader, a creusé un tunnel qui débouche non sur la mer mais dans la cellule de Dantès, et sauve Dantès qui se laissait mourir de faim. Edmond Dantès découvre alors en l'abbé Faria une personne exceptionnelle, érudit dans tous les domaines scientifiques et philosophiques. L'abbé Faria fait la lumière sur le complot machiné contre Edmond en lui montrant son écriture de la main gauche qui ressemble étrangement à celle de la lettre et en lui expliquant que M. Noirtier est en réalité le père de Villefort. Ces révélations instillent la vengeance dans le cœur d'Edmond. L'abbé Faria, très érudit, se lie d'amitié avec Dantès et lui donne une éducation exceptionnelle tant en économie qu'en sciences, en politique, en philosophie mais aussi en langues. Faria fait une première crise de catalepsie qui le laisse paralysé d'un bras et d'une jambe rendant ainsi l'évasion qu'ils préparaient impossible. Il lui fait alors part d'un secret qui le fait lui-même passer pour fou aux yeux de ses geôliers et, pendant un court moment, de Dantès : il est le dépositaire d'un immense trésor, celui des Spada, enfoui depuis des siècles dans l'île de Monte Cristo. Le vieux prêtre meurt d'un troisième accès et Edmond, pensant pouvoir s'échapper, prend la place du cadavre dans le linceul, en se munissant d'un couteau au cas où il serait découvert. Il comprend au dernier moment que tous les prisonniers morts en captivité sont jetés à la mer avec aux pieds un boulet de trente-six1 au château d'If et se libère grâce à son couteau. Sa captivité aura duré quinze ans. Edmond nage jusqu'à l'île de Tiboulen et est récupéré par un bateau de contrebandiers avec lesquels il noue des liens. C'est grâce à ce bateau, sur lequel il travaille temporairement comme marin, qu'il parvient à atteindre l'île de Monte-Cristo. Devenu très riche grâce au trésor des Spada dont il prend possession, Dantès retourne à Marseille où il apprend la mort de son père et constate la disparition de ses quatre « amis » : Danglars, Fernand, Caderousse et Villefort."

nlp = spacy.load("fr_core_news_sm")  #modèle français
txt = nlp(sample_txt) #adapte le texte au modèle français
txt_tokenized = [t.text for t in txt]
stop_words_french = nlp.Defaults.stop_words #liste de stopwords fr

In [ ]:
txt_token_stop = [t for t in txt_tokenized if t not in stop_words_french] #vire les "stopwords"

txt_lemmatized=[t.lemma_ for t in txt]#que lemmes

txt_lemma_stop = [t for t in txt_lemmatized if t not in stop_words_french]#lemmes sans les stopswords

txt_type=[(t.lemma_,t.tag_) for t in txt]
txt_type=[t for t in txt_type if t[0] not in stop_words_french]

In [ ]:
lemmatizer = FrenchLefffLemmatizer()
type_lemmatized_CC=[lemmatizer.lemmatize(t,'all') for t in txt_token_stop]
type_lemmatized_CC=[t[0] for t in type_lemmatized_CC if len(t)!=0]

txt_lemmatized_CC=[t[0] for t in type_lemmatized_CC]#que lemmes CC
txt_lemma_stop_CC=[t for t in txt_lemmatized_CC if t not in stop_words_french]#lemmes CC sans les stopwords

In [ ]:
def summary_tables (type,lemmatized) :   
    d_type=pd.DataFrame(data=type, columns=['Word','Type'])
    tk_count=Counter(lemmatized)
    d_count=pd.DataFrame.from_dict(data=tk_count,orient='index',columns=["Count"])
    d_count=d_count.reset_index()
    d_count=d_count.rename(columns={"index":"Word"})

    d_summary_order=d_type.rename_axis(index='Order')
    d_summary=d_type.merge(d_count,on='Word')
    d_summary=d_summary.sort_values('Count', ascending=False)
    d_summary=d_summary.reset_index(drop=True)
    d_summary=d_summary.drop_duplicates(subset=['Word'])
    return (d_summary,d_summary_order)

d_sum,d_sum_order=summary_tables(txt_type,txt_lemmatized)
d_sum_CC,d_sum_order_CC=summary_tables(type_lemmatized_CC,txt_lemmatized_CC)